# Intro to Spark DataFrames

dict and pandas

In [22]:
# store data by rows ( a row at a time) or by columns (a column at a time)

# columnar storage (data stored as columns). is not convineinet for access rows

# dictionary example
columnar_table = {"col1":[2,3,4],
                  "col2":[2.4,5.6,4.4]}

columnar_table["col1"]

[2, 3, 4]

In [23]:
# get first row, need to step each column 
columnar_table["col1"][0],columnar_table["col2"][0]

(2, 2.4)

In [24]:
# row storage (data stored as rows)
row_table = [{'col1':2,'col2':2.4},
             {'col1':3,'col2':5.6},
             {'col1':4,'col2':4.4}]

row_table[0]
# get first column, need to step each row, inefficient

{'col1': 2, 'col2': 2.4}

In [25]:
import pandas as pd
# pandas is columnar storage

pandas_df = pd.DataFrame(columnar_table)
pandas_df

,col1,col2
0,2,2.4
1,3,5.6
2,4,4.4


In [26]:
pandas_df = pd.DataFrame(row_table)
pandas_df

,col1,col2
0,2,2.4
1,3,5.6
2,4,4.4


In [27]:
pandas_df['col1'].values # it's a array

array([2, 3, 4])

In [28]:
pandas_df.iloc[[1]] # access row

,col1,col2
1,3,5.6


In [29]:
# serialization formats
# popular columnar formats
# parquet - tabular data as columns
# ORC

# popular row-based formats
# avro - tabular data as rows

In [30]:
#from pyspark import SparkContext

#sc = SparkContext("local","myapp")

In [1]:
# for DATAFRAMES!!

from pyspark.sql import SparkSession
# name of a computer, 7077 stardard port
# want one copy of my data

ss = SparkSession.builder.\
     master("spark://spark-master:7077").\
     config("fs.defaultFS", "hdfs://namenode:8020").\
     config("dfs.replication","1").\
     appName("myapp").getOrCreate()

## Create Manualy

In [2]:
from pyspark.sql import Row

df = ss.createDataFrame([Row(col1=3,col2=6.3),
                         Row(col1=4,col2=7.4),
                         Row(col1=5,col2=8.3)])

In [33]:
df.collect()

[Row(col1=3, col2=6.3), Row(col1=4, col2=7.4), Row(col1=5, col2=8.3)]

In [34]:
df.take(2) # grap two random rows

[Row(col1=3, col2=6.3), Row(col1=4, col2=7.4)]

In [35]:
df.head(2) # grap two first rows

[Row(col1=3, col2=6.3), Row(col1=4, col2=7.4)]

In [36]:
df.show()

+----+----+
|col1|col2|
+----+----+
|   3| 6.3|
|   4| 7.4|
|   5| 8.3|
+----+----+



## Create from pandas

In [37]:
df = ss.createDataFrame(pandas_df)

In [38]:
df.show()

+----+----+
|col1|col2|
+----+----+
|   2| 2.4|
|   3| 5.6|
|   4| 4.4|
+----+----+



## Create from an RDD

In [41]:
sc = ss.sparkContext
myrdd = sc.parallelize([(2,6.8),(3,4.6),(4,7.3)])

In [42]:
myrdd.collect()

[(2, 6.8), (3, 4.6), (4, 7.3)]

In [46]:
df = ss.createDataFrame(myrdd,["col1","col2"])

In [47]:
df.show()

+----+----+
|col1|col2|
+----+----+
|   2| 6.8|
|   3| 4.6|
|   4| 7.3|
+----+----+



## Create from list of tuples

In [56]:
df = ss.createDataFrame([(2,3.4),(3,6.7),(4,9.3)],["col1","col2"])

In [57]:
df.show()

+----+----+
|col1|col2|
+----+----+
|   2| 3.4|
|   3| 6.7|
|   4| 9.3|
+----+----+



## Convert DF to RDD

In [51]:
myrdd = df.rdd

In [52]:
myrdd.collect()

[Row(col1=2, col2=6.8), Row(col1=3, col2=4.6), Row(col1=4, col2=7.3)]

## Convert DF to pandas

In [58]:
pandas_df2 = df.toPandas()

In [60]:
pandas_df2.head() #it's on my local machine

,col1,col2
0,2,3.4
1,3,6.7
2,4,9.3


In [61]:
df.dtypes

[('col1', 'bigint'), ('col2', 'double')]

In [63]:
# sizes of integers: byte, short, int, bigint/long
# sizes of floats: float, double

## Schema

In [65]:
df.schema # telling you type is struct, inside of strcture you have a list; inside list is a field type...

StructType(List(StructField(col1,LongType,true),StructField(col2,DoubleType,true)))

In [66]:
from pyspark.sql.types import StructType, StructField, ByteType, DoubleType

In [67]:
schema = StructType([StructField("col1",ByteType(),True),
                     StructField("col2",DoubleType(),True)])
                                              #true: it's ok to have missing data

In [68]:
df_from_schema = ss.createDataFrame([(3,6.23),(5,4.2)],schema)

In [70]:
df_from_schema.schema # force the type to be the defalut

StructType(List(StructField(col1,ByteType,true),StructField(col2,DoubleType,true)))

## Writing out

In [72]:
sc.sparkUser()

'luna0212'

In [3]:
from hdfs import InsecureClient
client = InsecureClient("http://namenode:50070",user='vagrant')

In [74]:
client.list('/')

['User', 'Users']

In [4]:
client.list('/Users')

['vagrant']

In [5]:
client.delete('/Users/vagrant/mydf.csv', recursive=True)

False

In [6]:
df.write.csv('hdfs://namenode/Users/vagrant/mydf.csv') # write into a directory, has two csv (two partition)

Py4JJavaError: An error occurred while calling o58.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scal

In [78]:
df.write.parquet('hdfs://namenode/User/vagrant/test_df_.parquet')

Py4JJavaError: An error occurred while calling o268.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [79]:
myrdd.saveAsPickleFile('hdfs://namenode/User/vagrant/myrdd.pickle')

Py4JJavaError: An error occurred while calling o312.saveAsObjectFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions